In [35]:
!pip install transformers==4.10.0

  Using cached transformers-4.10.0-py3-none-any.whl.metadata (51 kB)
  Using cached tokenizers-0.10.3-cp39-cp39-win_amd64.whl.metadata (5.8 kB)
Using cached transformers-4.10.0-py3-none-any.whl (2.8 MB)
Using cached tokenizers-0.10.3-cp39-cp39-win_amd64.whl (2.0 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.19.1
    Uninstalling tokenizers-0.19.1:
      Successfully uninstalled tokenizers-0.19.1
  Attempting uninstall: transformers
    Found existing installation: transformers 4.41.2
    Uninstalling transformers-4.41.2:
      Successfully uninstalled transformers-4.41.2


  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
sentence-transformers 3.0.0 requires transformers<5.0.0,>=4.34.0, but you have transformers 4.10.0 which is incompatible.


In [1]:
import sys
print(sys.version)

3.9.18 (main, Sep 11 2023, 14:09:26) [MSC v.1916 64 bit (AMD64)]


In [2]:
pip show transformers

Name: transformersNote: you may need to restart the kernel to use updated packages.

Version: 4.10.0
Summary: State-of-the-art Natural Language Processing for TensorFlow 2.0 and PyTorch
Home-page: https://github.com/huggingface/transformers
Author: Thomas Wolf, Lysandre Debut, Victor Sanh, Julien Chaumond, Sam Shleifer, Patrick von Platen, Sylvain Gugger, Suraj Patil, Stas Bekman, Google AI Language Team Authors, Open AI team Authors, Facebook AI Authors, Carnegie Mellon University Authors
Author-email: thomas@huggingface.co
License: Apache
Location: d:\anaconda\envs\py39\lib\site-packages
Requires: filelock, huggingface-hub, numpy, packaging, pyyaml, regex, requests, sacremoses, tokenizers, tqdm
Required-by: sentence-transformers


In [1]:
import pandas as pd
import numpy as np
import re
import demoji

# Load dataset
df = pd.read_csv("D:\\Sinophobia Sentiment Analysis\\archive\\Tweets_India\\Tweets_India\\India.csv", low_memory=False)


In [2]:
df

,Unnamed: 0,id,created_at,text,user_location
0,42,1312972291332952064,Mon Oct 05 04:26:02 +0000 2020,"#India's #COVID19 tally crosses 66-lakh. 74,4...",India
1,149,1312972302355558402,Mon Oct 05 04:26:05 +0000 2020,@narendramodi Talk something about mass unempl...,Mumbai
2,162,1312972309032955906,Mon Oct 05 04:26:06 +0000 2020,RT @hiteshi75: When law is supposed to be simi...,"Bihar, India"
3,260,1312972316142256128,Mon Oct 05 04:26:08 +0000 2020,RT @Sowmyareddyr: Oh did we miss any plate ban...,"Bengaluru, India"
4,402,1312972335188647937,Mon Oct 05 04:26:13 +0000 2020,"RT @parthomishra: Amid #COVID19, teachers have...","Karimnagar,Telangana,India"
...,...,...,...,...,...
5411289,888026,Tue Mar 01 01:18:30 +0000 2022,@LiveLawIndia @Shrutikakk @DoptSecretary relie...,"New Delhi, India",NaN
5411290,888205,Tue Mar 01 01:18:50 +0000 2022,Now I see why CDC rushed to ditch masks on Fri...,"Hyderabad, Telangana",NaN
5411291,888775,Tue Mar 01 01:19:35 +0000 2022,Lockdown Me Bhi Majdoor Paidal Aaye Ab Ukrain ...,"Samastipur, India",NaN
5411292,888808,Tue Mar 01 01:19:42 +0000 2022,#UnitedNations FAILED\nIndia-China conflict- U...,India,NaN


In [3]:
# Data cleaning
df = df.drop(columns=['Unnamed: 0', 'id', 'user_location'])

df = df.dropna(axis=0)


In [4]:
df

,created_at,text
0,Mon Oct 05 04:26:02 +0000 2020,"#India's #COVID19 tally crosses 66-lakh. 74,4..."
1,Mon Oct 05 04:26:05 +0000 2020,@narendramodi Talk something about mass unempl...
2,Mon Oct 05 04:26:06 +0000 2020,RT @hiteshi75: When law is supposed to be simi...
3,Mon Oct 05 04:26:08 +0000 2020,RT @Sowmyareddyr: Oh did we miss any plate ban...
4,Mon Oct 05 04:26:13 +0000 2020,"RT @parthomishra: Amid #COVID19, teachers have..."
...,...,...
5411289,@LiveLawIndia @Shrutikakk @DoptSecretary relie...,"New Delhi, India"
5411290,Now I see why CDC rushed to ditch masks on Fri...,"Hyderabad, Telangana"
5411291,Lockdown Me Bhi Majdoor Paidal Aaye Ab Ukrain ...,"Samastipur, India"
5411292,#UnitedNations FAILED\nIndia-China conflict- U...,India


In [5]:
# Initialize an empty list to hold DataFrames
dfs_to_concat = []

# List of sinophobic terms
sinophobic_terms = ['china', 'chinese', 'sinophobia','sinophobic', 'prc', 'wuhan', 'hubei', 'beijing', 'kung flu', 'chn','cn', 'ccp', 'yellow peril', 'chink', 'chinks', 
                    'chingchong', 'ching chong', 'gook', 'chyna', 'mainland', 'mainlander', 'bugland', 'chines', 
                    'mainla', 'chinazi', 'bugmen', 'chankoro', 'insectoid']

for term in sinophobic_terms:
    filtered_df = df[df['text'].str.contains(f'\\b{term}\\b', case=False, na=False, regex=True)]
    print(f"Term: {term}, Count: {len(filtered_df)}")
    print(filtered_df['text'].head(5))  # Print the first 5 rows to inspect the content
    dfs_to_concat.append(filtered_df)


# Concatenate all filtered DataFrames
final = pd.concat(dfs_to_concat, ignore_index=True)

# Remove duplicate entries
final = final.drop_duplicates(subset=['text'])

# Reset the index of the final DataFrame
final = final.reset_index(drop=True)

# Display the first few rows of the final DataFrame
final.head()

Term: china, Count: 11001
243    RT @KenRoth: Percy Cradock once said: "China’s...
357    @jenniferatntd @realDonaldTrump It's Wuhan vir...
561    RT @omasotra: Most coronavirus deaths\n\nItaly...
635    Air Marshal Amit Dev takes charge of Eastern A...
753    RT @IndoPac_Info: #China’s tech giants face ‘n...
Name: text, dtype: object
Term: chinese, Count: 4946
120    RT @republic: COVID-19 vaccine: Chinese compan...
202    @Groot95606268 @oppomobileindia this video was...
521    @wokebhakt One does not realise the seriousnes...
753    RT @IndoPac_Info: #China’s tech giants face ‘n...
876    RT @WIONews: Chinese drugmaker Sinovac, which ...
Name: text, dtype: object
Term: sinophobia, Count: 2
128610     While the western media and pundits remain obl...
3604624    Overall, we find a rise in hate sentiments tow...
Name: text, dtype: object
Term: sinophobic, Count: 0
Series([], Name: text, dtype: object)
Term: prc, Count: 71
75249     RT @JMichaelCole1: My survey of the PRC's stra...
8377

,created_at,text
0,Mon Oct 05 04:32:04 +0000 2020,"RT @KenRoth: Percy Cradock once said: ""China’s..."
1,Mon Oct 05 04:35:06 +0000 2020,@jenniferatntd @realDonaldTrump It's Wuhan vir...
2,Mon Oct 05 04:40:07 +0000 2020,RT @omasotra: Most coronavirus deaths\n\nItaly...
3,Mon Oct 05 04:41:45 +0000 2020,Air Marshal Amit Dev takes charge of Eastern A...
4,Mon Oct 05 04:44:35 +0000 2020,RT @IndoPac_Info: #China’s tech giants face ‘n...


In [6]:
len(final)

7191

In [7]:
final.to_csv("D:\\Sinophobia Sentiment Analysis\\NewFiltering\\InAfterFilter.csv")

In [8]:
df_withtime = final

In [9]:
final = final.drop(columns=['created_at'])

In [10]:
contractionsWithAnotherInvertedComma = {
"ain’t": "am not", "aren’t": "are not", "can’t": "cannot", "can’t’ve": "cannot have", "’cause": "because", "could’ve": "could have", "couldn’t": "could not",
"couldn’t’ve": "could not have", "didn’t": "did not", "doesn’t": "does not", "don’t": "do not", "hadn’t": "had not", "hadn’t’ve": "had not have",
"hasn’t": "has not", "haven’t": "have not", "he’d": "he had", "he’d’ve": "he would have", "he’ll": "he will", "he’ll’ve": "he will have", "he’s": "he is",
"how’d": "how did", "how’d’y": "how do you", "how’ll": "how will", "how’s": "how is", "i’d": "i would", "i’d’ve": "i would have",
"i’ll": "i will", "i’ll’ve": "i will have", "i’m": "i am", "i’ve": "i have", "isn’t": "is not", "it’d": "it would",
"it’d’ve": "it would have", "it’ll": "it will", "it’ll’ve": "it will have", "it’s": "it is", "let’s": "let us",
"ma’am": "madam", "mayn’t": "may not", "might’ve": "might have", "mightn’t": "might not", "mightn’t’ve": "might not have", "must’ve": "must have", "mustn’t": "must not",
"mustn’t’ve": "must not have", "needn’t": "need not", "needn’t’ve": "need not have", "o’clock": "of the clock", "oughtn’t": "ought not", "oughtn’t’ve": "ought not have",
"shan’t": "shall not", "shan’t’ve": "shall not have", "she’d": "she would", "she’d’ve": "she would have", "she’ll": "she will",
"she’ll’ve": "she will have", "she’s": "she is", "should’ve": "should have", "shouldn’t": "should not", "shouldn’t’ve": "should not have",
"so’ve": "so have", "so’s": "so is", "that’d": "that would", "that’d’ve": "that would have", "that’s": "that is", "there’d": "there would",
"there’d’ve": "there would have", "there’s": "there is", "they’d": "they would", "they’d’ve": "they would have", "they’ll": "they will",
"they’ll’ve": "they will have", "they’re": "they are", "they’ve": "they have", "to’ve": "to have", "wasn’t": "was not", "we’d": "we would",
"we’d’ve": "we would have", "we’ll": "we will", "we’ll’ve": "we will have", "we’re": "we are", "we’ve": "we have", "weren’t": "were not", "what’ll": "what will",
"what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is",
"when’ve": "when have", "where’d": "where did", "where’s": "where is", "where’ve": "where have", "who’ll": "who will", "who’ll’ve": "who will have",
"who’s": "who is", "who’ve": "who have", "why’s": "why is", "why’ve": "why have", "will’ve": "will have", "won’t": "will not", "won’t’ve": "will not have",
"would’ve": "would have", "wouldn’t": "would not", "wouldn’t’ve": "would not have", "y’all": "you all", "y’all’d": "you all would", "y’all’d’ve": "you all would have",
"y’all’re": "you all are", "y’all’ve": "you all have", "you’d": "you would", "you’d’ve": "you would have", "you’ll": "you will", "you’ll’ve": "you will have",
"you’re": "you are", "you’ve": "you have"
}

In [11]:
contractions = {
"ain't": "am not", "aren't": "are not", "can't": "cannot", "can't've": "cannot have", "'cause": "because", "could've": "could have", "couldn't": "could not",
"couldn't've": "could not have", "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hadn't've": "had not have",
"hasn't": "has not", "haven't": "have not", "he'd": "he had", "he'd've": "he would have", "he'll": "he will", "he'll've": "he will have", "he's": "he is",
"how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is", "i'd": "i would", "i'd've": "i would have",
"i'll": "i will", "i'll've": "i will have", "i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",
"it'd've": "it would have", "it'll": "it will", "it'll've": "it will have", "it's": "it is", "let's": "let us",
"ma'am": "madam", "mayn't": "may not", "might've": "might have", "mightn't": "might not", "mightn't've": "might not have", "must've": "must have", "mustn't": "must not",
"mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have", "o'clock": "of the clock", "oughtn't": "ought not", "oughtn't've": "ought not have",
"shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have", "she'd": "she would", "she'd've": "she would have", "she'll": "she shall / she will",
"she'll've": "she will have", "she's": "she is", "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have",
"so've": "so have", "so's": "so is", "that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",
"there'd've": "there would have", "there's": "there is", "they'd": "they would", "they'd've": "they would have", "they'll": "they will",
"they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have", "wasn't": "was not", "we'd": "we would",
"we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are", "we've": "we have", "weren't": "were not", "what'll": "what will",
"what'll've": "what will have", "what're": "what are", "what's": "what is", "what've": "what have", "when's": "when is",
"when've": "when have", "where'd": "where did", "where's": "where is", "where've": "where have", "who'll": "who will", "who'll've": "who will have",
"who's": "who is", "who've": "who have", "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",
"would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all", "y'all'd": "you all would", "y'all'd've": "you all would have",
"y'all're": "you all are", "y'all've": "you all have", "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",
"you're": "you are", "you've": "you have"
}

In [12]:
abbreviations_dict = {
    "ccp": "chinese communist party", "cn": "china", "chn": "china", "prc": "peoples republic of china",
    "btw": "by the way", "brb": "be right back",
    "b4n": "bye for now",
    "fyi": "for your information",
    "ilu": "i love you",
    "iow": "in other words",
    "roflol": "rolling on the floor laughing out loud",
    "rotflmao": "rolling on the floor laughing my ass off",
    "fb": "facebook", "ig": "instagram", "li": "linkedin", "yt": "youtube", "tw": "twitter",
    "dm": "direct message", "mt": "modified tweet", "pm": "private message", "rt": "retweet",
    "b2b": "business to business", "b2c": "business to consumer", "cmgr": "community manager",
    "cms": "content management system", "cpc": "cost per click", "cpm": "cost per thousand impressions",
    "cr": "conversion rate", "cro": "conversion rate optimization", "cta": "call to action",
    "ctr": "click-through rate", "roi": "return on investment", "smb": "small and midsize businesses",
    "smp": "social media platform", "smm": "social media marketing", "smo": "social media optimization",
    "solomo": "social, local and mobile", "srp": "social relationship platform", "tos": "terms of service",
    "ugc": "user-generated content", "api": "application programming interface", "cx": "customer experience",
    "esp": "email service provider", "ga": "google analytics", "isp": "internet service provider",
    "pv": "page views", "rss": "really simple syndication", "saas": "software as a service",
    "sem": "search engine marketing", "seo": "search engine optimization", "sov": "share of voice",
    "ui": "user interface", "url": "uniform resource locator", "uv": "unique views", "ux": "user experience",
    "afaik": "as far as i know", "ama": "ask me anything", "btaim": "be that as it may",
    "bts": "behind the scenes", "dae": "does anyone else", "dyk": "did you know",
    "eli5": "explain like i’m five", "fbf": "flashback friday", "fbo": "facebook official",
    "ff": "follow friday", "fomo": "fear of missing out", "ftfy": "fixed that for you",
    "ftw": "for the win", "g2g": "got to go", "gg": "good game", "gtr": "got to run",
    "hbd": "happy birthday", "hifw": "how i feel when", "hmb": "hit me back", "hmu": "hit me up",
    "ht": "hat tip", "h/t": "hat tip", "hth": "here to help", "icymi": "in case you missed it",
    "idc": "i don’t care", "idk": "i don’t know", "ikr": "i know, right?", "ily": "i love you",
    "imho": "in my humble opinion", "imo": "in my opinion", "irl": "in real life", "jk": "just kidding",
    "lmao": "laughing my ass off", "lmk": "let me know", "lms": "like my status",
    "lol": "laughing out loud", "mcm": "man crush monday", "mfw": "my face when",
    "mtfbwy": "may the force be with you", "nbd": "no big deal", "nm": "not much",
    "nsfw": "not safe for work", "nvm": "never mind", "oh": "overheard", "omw": "on my way",
    "ootd": "outfit of the day", "op": "original poster", "otp": "one true pairing", "ppl": "people",
    "rofl": "rolling on the floor laughing", "roflmao": "rolling on the floor laughing my ass off",
    "sfw": "safe for work", "smh": "shaking my head", "tbh": "to be honest", "tbbh": "to be brutally honest",
    "tbt": "throwback thursday", "tfw": "that feeling when", "tgif": "thank god it’s friday",
    "til": "today i learned", "tl;dr": "too long; didn’t read", "tmi": "too much information",
    "wbu": "what about you?", "wbw": "way back wednesday", "wfh": "work from home", "yolo": "you only live once",
    "afk": "away from keyboard", "asap": "as soon as possible", "atk": "at the keyboard",
    "atm": "at the moment", "a3": "anytime, anywhere, anyplace", "bak": "back at keyboard",
    "bbl": "be back later", "bbs": "be back soon", "bfn": "bye for now", "brt": "be right there",
    "b4": "before", "cu": "see you", "cul8r": "see you later", "cya": "see you", "faq": "frequently asked questions",
    "fc": "fingers crossed", "fwiw": "for what it's worth", "gal": "get a life", "gn": "good night",
    "gmta": "great minds think alike", "gr8": "great!", "g9": "genius", "ic": "i see", "icq": "i seek you",
    "kiss": "keep it simple, stupid", "ldr": "long distance relationship", "ltns": "long time no see",
    "l8r": "later", "mte": "my thoughts exactly", "m8": "mate", "nrn": "no reply necessary", "oic": "oh i see",
    "pita": "pain in the a..", "prt": "party", "prw": "parents are watching", "qpsa": "que pasa?",
    "sk8": "skate", "stats": "your sex and age", "asl": "age, sex, location", "thx": "thank you",
    "ttfn": "ta-ta for now!", "ttyl": "talk to you later", "u": "you", "u2": "you too", "u4e": "yours for ever",
    "wb": "welcome back", "wtf": "what the f...", "wtg": "way to go!", "wuf": "where are you from?",
    "w8": "wait...", "7k": "sick:-d laugher"
}


In [13]:
# Preprocessing
class preprocess():
    def __init__(self, df, contractions, otherContractions):
        self.df = df
        self.contractions = contractions
        self.otherContractions = otherContractions

    def lower(self, tweet):
        return tweet.lower()

    def abbreviate(self, tweet):
        tweet_tokens = tweet.split(' ')
        for i, token in enumerate(tweet_tokens):
            cleaned_token = re.sub('[^a-zA-Z0-9-_.]', '', token)
            if cleaned_token.lower() in abbreviations_dict:
                tweet_tokens[i] = abbreviations_dict[cleaned_token.lower()]
        return ' '.join(tweet_tokens)

    def expand(self, tweet):
        for word in tweet.split():
            if word in self.contractions.keys():
                tweet = tweet.replace(word, self.contractions[word])
            elif word in self.otherContractions.keys():
                tweet = tweet.replace(word, self.otherContractions[word])
        return tweet

    def emoji2text(self, tweet):
        emojis = demoji.findall(tweet)
        new_tweet = []
        for word in tweet.split():
            if word in emojis.keys():
                tweet = tweet.replace(word, emojis[word])
                new_tweet.append(emojis[word])
            wordmojis = demoji.findall(word)
            for char in word:
                if char in wordmojis.keys():
                    tweet = tweet.replace(word, wordmojis[char])

        return tweet

    def remove_hashtags(self, tweet):
        return re.sub(r'\#w+', '', tweet)

    def remove_mentions(self, tweet):
        for word in tweet.split():
            if word[0] == '@':
                tweet = tweet.replace(word, '')
        return tweet

    def remove_punctuations(self, tweet):
        punct = string.punctuation
        trantab = str.maketrans(punct, len(punct)*' ')
        return tweet.translate(trantab)

    def remove_url(self, tweet):
        return re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', '', tweet, flags = re.MULTILINE)

    def preprocess_tweet(self, tweet):
        tweet = self.lower(tweet)
        tweet = self.abbreviate(tweet)
        tweet = self.expand(tweet)
        tweet = self.emoji2text(tweet)
        tweet = self.remove_mentions(tweet)
        tweet = self.remove_url(tweet)
        tweet = self.remove_hashtags(tweet)
        tweet = self.remove_punctuations(tweet)

        return tweet

In [14]:
import csv
import string
pp_class = preprocess(final, contractions, contractionsWithAnotherInvertedComma)
final['text'] = final['text'].apply(lambda x : pp_class.preprocess_tweet(x))

In [15]:
def bruteGen(tweet):
    tweet = tweet.replace("indiavscorona", "india versus coronavirus")
    tweet = tweet.replace("outbreakindia", "outbreak india")
    tweet = tweet.replace("real”", "real")
    tweet = tweet.replace("mutra", "urine")
    tweet = tweet.replace("fakenews", "fake news")
    tweet = tweet.replace("“omg", "oh my god")
    tweet = tweet.replace("“damn", "damn")
    tweet = tweet.replace("god’s", "gods")
    tweet = tweet.replace("lockdownextension", "lockdown extension")
    tweet = tweet.replace("कोरोना", "coronavirus")
    tweet = tweet.replace("indiathanks", "india thanks")
    tweet = tweet.replace("coronacoronavirus", "coronavirus")
    tweet = tweet.replace('coronavirusinsa', "coronavirus in south africa")
    tweet = tweet.replace('coronaviruscanada', 'coronavirus canada')
    tweet = tweet.replace('coronavirusau', 'coronavirus australia')
    tweet = tweet.replace('coronavirusaus', 'coronavirus australia')
    tweet = tweet.replace('cuomoprimetime', 'new york governor prime time')
    tweet = tweet.replace('letsfightcoronavirus', 'let us fight coronavirus')
    tweet = tweet.replace("covid 19", "coronavirus")
    tweet = tweet.replace("covid19", "coronavirus")
    tweet = tweet.replace("covid", "coronavirus")
    tweet = tweet.replace("aprilfoolsday", "april fools day")
    tweet = tweet.replace("covid-19", "coronavirus")
    tweet = tweet.replace("stayathome", "stay at home")
    tweet = tweet.replace("“april", "april")
    tweet = tweet.replace("“i", "i")
    tweet = tweet.replace("aprilfools", "april fools")
    tweet = tweet.replace("coronavirusoutbreak", "coronavirus outbreak")
    tweet = tweet.replace("virus-19", "coronavirus")
    tweet = tweet.replace("fool’s", "fools")
    tweet = tweet.replace("what’s", "what is")
    tweet = tweet.replace("coronavirus”", "coronavirus")
    tweet = tweet.replace("fools”", "fools")
    tweet = tweet.replace("stayhome", "stay home")
    tweet = tweet.replace("quarantinelife", "quarantine life")
    tweet = tweet.replace("tablighijamaat", "muslims")
    tweet = tweet.replace("corona”", "coronavirus")
    tweet = tweet.replace("fauci", "physician")
    tweet = tweet.replace("april’s", "april")
    tweet = tweet.replace("pmkcallscurfewextension", "prime minister calls for curfew extension")
    tweet = tweet.replace("“virus", "coronavirus")
    tweet = tweet.replace("virus”", "coronavirus")
    tweet = tweet.replace("“corona", "coronavirus")
    tweet = tweet.replace("coronavirustruth", "coronavirus truth")
    tweet = tweet.replace("socialdistancing", "social distancing")
    tweet = tweet.replace("homestaysafe", "home stay safe")
    tweet = tweet.replace("“coronavirus", "coronavirus")
    tweet = tweet.replace("coronavirusupdate", "coronavirus update")
    tweet = tweet.replace("virusvirus", "coronavirus")
    tweet = tweet.replace("coronaviruspandemic", "coronavirus pandemic")
    tweet = tweet.replace("thelockdown", "the lockdown")
    tweet = tweet.replace("nizamuddin", "delhi")
    tweet = tweet.replace("trump’s", "donald trump")
    tweet = tweet.replace("“the", "the")
    tweet = tweet.replace("virus2019", "coronavirus")
    tweet = tweet.replace("indiafightscorona", "india fights coronavirus")
    tweet = tweet.replace("homesavelives", "home save lives")
    tweet = tweet.replace("everyone’s", "everyone")
    tweet = tweet.replace("coronariskforprisoners", "coronavirus risk for prisoners")
    tweet = tweet.replace("coronavirususa", "coronavirus usa")
    tweet = tweet.replace("tablighi", "mosque")
    tweet = tweet.replace("delhimarkaz", "delhi mosque")
    tweet = tweet.replace("coronajihad", "coronavirus struggle")
    tweet = tweet.replace("coronajihaad", "coronavirus struggle")
    tweet = tweet.replace("aprilfool", "april fool")
    tweet = tweet.replace("trumppressconference", "trump press conference")
    tweet = tweet.replace("i’m", "i am")
    tweet = tweet.replace("tigerking", "tiger king")
    tweet = tweet.replace("it’s", "it is")
    tweet = tweet.replace("trumpvirus", "trump virus")
    tweet = tweet.replace("today’s", "today is")
    tweet = tweet.replace("“you", "you")
    tweet = tweet.replace("“a", "a")
    tweet = tweet.replace("fools’", "fools")
    tweet = tweet.replace("rtgnews", "news")
    tweet = tweet.replace("19india", "india")
    tweet = tweet.replace("coronavirusindia", "coronavirus india")
    tweet = tweet.replace("y’all", "you all")
    tweet = tweet.replace("मीडिया", "media")
    tweet = tweet.replace("here’s", "here is")
    tweet = tweet.replace("“we", "we")
    tweet = tweet.replace("“fuck", "fuck")
    tweet = tweet.replace("flattenthecurve", "flatten the curve")
    tweet = tweet.replace("jammuandkashmir", "jammu and kashmir")
    tweet = tweet.replace("chriscuomo", "new york governor")
    tweet = tweet.replace("‘april", "april")
    tweet = tweet.replace("dranbumani", "doctor")
    tweet = tweet.replace("tndemandsmasstesting", "tamil nadu demands mass testing")
    tweet = tweet.replace("tabligi", "muslims")
    tweet = tweet.replace("don’t", "do not")
    tweet = tweet.replace("वायरस", "virus")
    tweet = tweet.replace("letsfightvirus", "let us fight virus")
    return tweet


In [16]:
final['text'] = final['text'].apply(lambda x : bruteGen(x))

In [17]:
final.head

<bound method NDFrame.head of                                                    text
0     retweet  percy cradock once said   china’s lea...
1       it is wuhan virus which chinese communist pa...
2     retweet  most coronavirus deaths\n\nitaly  340...
3     air marshal amit dev takes charge of eastern a...
4     retweet   china’s tech giants face ‘new busine...
...                                                 ...
7186  retweet  i remember last year all bhakts were ...
7187  retweet  i repeat my tweet  why is the world c...
7188     that is the work we need from s  d  m   int...
7189  retweet  while  chinazi is sitting back after ...
7190  retweet  a very comforting news  we need to ge...

[7191 rows x 1 columns]>

In [18]:
final.to_csv("D:\\Sinophobia Sentiment Analysis\\NewFiltering\\InAfterpreprocess.csv")

In [19]:
labels = ['Optimistic', 'Thankful', 'Empathetic', 'Pessimistic',
       'Anxious', 'Sad', 'Annoyed', 'Denial', 'Official report', 'Joking']

In [20]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')


In [21]:
df = pd.DataFrame()
df['Tweet'] = final['text']
values = [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]] * len(final)
df[labels] = values

In [22]:
df.reset_index(drop=True, inplace=True)
df

,Tweet,Optimistic,Thankful,Empathetic,Pessimistic,Anxious,Sad,Annoyed,Denial,Official report,Joking
0,retweet percy cradock once said china’s lea...,0,0,0,0,0,0,0,0,0,0
1,it is wuhan virus which chinese communist pa...,0,0,0,0,0,0,0,0,0,0
2,retweet most coronavirus deaths\n\nitaly 340...,0,0,0,0,0,0,0,0,0,0
3,air marshal amit dev takes charge of eastern a...,0,0,0,0,0,0,0,0,0,0
4,retweet china’s tech giants face ‘new busine...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
7186,retweet i remember last year all bhakts were ...,0,0,0,0,0,0,0,0,0,0
7187,retweet i repeat my tweet why is the world c...,0,0,0,0,0,0,0,0,0,0
7188,that is the work we need from s d m int...,0,0,0,0,0,0,0,0,0,0
7189,retweet while chinazi is sitting back after ...,0,0,0,0,0,0,0,0,0,0


In [23]:
import torch
class CustomDataset(torch.utils.data.Dataset):
   def __init__(self,df,tokenizer,max_len):
     self.df = df
     self.tokenizer = tokenizer
     self.max_len = max_len
     self.title = self.df['Tweet']
     self.targets = self.df[labels].values

   def __len__(self):
     return len(self.title)

   def __getitem__(self,index):
     title = str(self.title[index])
     title = " ".join(title.split())

     inputs = self.tokenizer.encode_plus(
         title,
         None,
         add_special_tokens=True,
         max_length=self.max_len,
         padding='max_length',
         return_token_type_ids=True,
         truncation=True,
         return_attention_mask=True,
         return_tensors='pt'
     )
     return {
         'input_ids':inputs['input_ids'].flatten(),
         'attention_mask':inputs['attention_mask'].flatten(),
         'token_type_ids':inputs['token_type_ids'].flatten(),
         'targets':torch.FloatTensor(self.targets[index])
     }

In [24]:
MAX_LEN=256
_dataset = CustomDataset(df, tokenizer, MAX_LEN)
_data_loader = torch.utils.data.DataLoader(
    _dataset,
    shuffle=False,
    batch_size=8,
    num_workers=0
)

In [25]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [26]:
class BERT(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = BertModel.from_pretrained('bert-base-uncased')
        self.layer2 = torch.nn.Dropout(0.3)
        self.layer3 = torch.nn.Linear(768, 10)

    def forward(self, ids, mask, token_type_ids, return_dict = False):
        unw, out_1 = self.layer1(ids, attention_mask = mask, token_type_ids = token_type_ids)[0], self.layer1(ids, attention_mask = mask, token_type_ids = token_type_ids)[1]
        out_2 = self.layer2(out_1)
        out_final = self.layer3(out_2)
        return out_final


model = BERT()
model.to(device)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BERT(
  (layer1): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=Tru

In [27]:
model = torch.load("D:\\Sinophobia Sentiment Analysis\\Transformers4.10Bertmodel2.pth")

In [28]:
model.eval()
bert_outputs = []
with torch.no_grad():

  for index, batch in enumerate(_data_loader):
         input_ids = batch['input_ids'].to(device, dtype=torch.long)
         attention_mask = batch['attention_mask'].to(device, dtype=torch.long)
         token_type_ids = batch['token_type_ids'].to(device, dtype=torch.long)
         targets =  batch['targets'].to(device, dtype=torch.float)
         outputs = model(input_ids, attention_mask, token_type_ids)
         bert_outputs.extend(torch.sigmoid(outputs).cpu().detach().numpy().tolist())

In [29]:
test_outputs = np.array(bert_outputs)

for i in range(test_outputs.shape[0]):
    for j in range(test_outputs.shape[1]):
        if test_outputs[i][j] >= 0.5: test_outputs[i][j] = 1
        else: test_outputs[i][j] = 0

In [30]:
df[labels] = test_outputs.astype(int)

In [31]:
df

,Tweet,Optimistic,Thankful,Empathetic,Pessimistic,Anxious,Sad,Annoyed,Denial,Official report,Joking
0,retweet percy cradock once said china’s lea...,0,0,0,0,0,0,1,1,0,0
1,it is wuhan virus which chinese communist pa...,0,0,0,0,0,0,1,1,0,0
2,retweet most coronavirus deaths\n\nitaly 340...,0,0,0,0,0,0,0,0,1,0
3,air marshal amit dev takes charge of eastern a...,0,0,0,0,0,0,0,0,1,0
4,retweet china’s tech giants face ‘new busine...,0,0,0,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
7186,retweet i remember last year all bhakts were ...,1,0,0,0,0,0,0,0,0,1
7187,retweet i repeat my tweet why is the world c...,0,0,0,0,0,0,1,0,0,1
7188,that is the work we need from s d m int...,1,0,0,0,0,0,0,0,0,0
7189,retweet while chinazi is sitting back after ...,1,0,0,0,0,0,1,0,0,0


In [32]:
df_withtime.reset_index(drop=True, inplace=True)
df_withtime

,created_at,text
0,Mon Oct 05 04:32:04 +0000 2020,"RT @KenRoth: Percy Cradock once said: ""China’s..."
1,Mon Oct 05 04:35:06 +0000 2020,@jenniferatntd @realDonaldTrump It's Wuhan vir...
2,Mon Oct 05 04:40:07 +0000 2020,RT @omasotra: Most coronavirus deaths\n\nItaly...
3,Mon Oct 05 04:41:45 +0000 2020,Air Marshal Amit Dev takes charge of Eastern A...
4,Mon Oct 05 04:44:35 +0000 2020,RT @IndoPac_Info: #China’s tech giants face ‘n...
...,...,...
7186,Thu Apr 15 08:36:21 +0000 2021,RT @klassyclicks: I remember last year all bha...
7187,Mon May 03 06:38:13 +0000 2021,"RT @buzzindelhi: I repeat my tweet, why is the..."
7188,Mon May 17 05:45:40 +0000 2021,@SdmVeraval @collectorgirsom @pkumarias That's...
7189,Mon Oct 05 09:48:57 +0000 2020,RT @reachind_uk: While #Chinazi is sitting bac...


In [33]:
In = pd.DataFrame()
In['Tweet'] = df['Tweet']
In['text'] = df_withtime['text']
In['date'] = df_withtime['created_at']
In[labels] = df[labels]
In

,Tweet,text,date,Optimistic,Thankful,Empathetic,Pessimistic,Anxious,Sad,Annoyed,Denial,Official report,Joking
0,retweet percy cradock once said china’s lea...,"RT @KenRoth: Percy Cradock once said: ""China’s...",Mon Oct 05 04:32:04 +0000 2020,0,0,0,0,0,0,1,1,0,0
1,it is wuhan virus which chinese communist pa...,@jenniferatntd @realDonaldTrump It's Wuhan vir...,Mon Oct 05 04:35:06 +0000 2020,0,0,0,0,0,0,1,1,0,0
2,retweet most coronavirus deaths\n\nitaly 340...,RT @omasotra: Most coronavirus deaths\n\nItaly...,Mon Oct 05 04:40:07 +0000 2020,0,0,0,0,0,0,0,0,1,0
3,air marshal amit dev takes charge of eastern a...,Air Marshal Amit Dev takes charge of Eastern A...,Mon Oct 05 04:41:45 +0000 2020,0,0,0,0,0,0,0,0,1,0
4,retweet china’s tech giants face ‘new busine...,RT @IndoPac_Info: #China’s tech giants face ‘n...,Mon Oct 05 04:44:35 +0000 2020,0,0,0,1,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
7186,retweet i remember last year all bhakts were ...,RT @klassyclicks: I remember last year all bha...,Thu Apr 15 08:36:21 +0000 2021,1,0,0,0,0,0,0,0,0,1
7187,retweet i repeat my tweet why is the world c...,"RT @buzzindelhi: I repeat my tweet, why is the...",Mon May 03 06:38:13 +0000 2021,0,0,0,0,0,0,1,0,0,1
7188,that is the work we need from s d m int...,@SdmVeraval @collectorgirsom @pkumarias That's...,Mon May 17 05:45:40 +0000 2021,1,0,0,0,0,0,0,0,0,0
7189,retweet while chinazi is sitting back after ...,RT @reachind_uk: While #Chinazi is sitting bac...,Mon Oct 05 09:48:57 +0000 2020,1,0,0,0,0,0,1,0,0,0


In [34]:
In.to_csv("D:\\Sinophobia Sentiment Analysis\\NewFiltering\\Result\\In.csv")